In [1]:
import numpy as np
import pandas as pd
import lxml.etree as etree
import sqlite3 as sql
import matplotlib.pyplot as plt

In [11]:
db = sql.connect('../data/database.sqlite')

In [24]:
df_match = pd.read_sql_query('SELECT * FROM Match', db)
df_team = pd.read_sql_query('SELECT * FROM Team', db)

In [249]:
t = [print(x) for x in df_match.columns]

id
country_id
league_id
season
stage
date
match_api_id
home_team_api_id
away_team_api_id
home_team_goal
away_team_goal
home_player_X1
home_player_X2
home_player_X3
home_player_X4
home_player_X5
home_player_X6
home_player_X7
home_player_X8
home_player_X9
home_player_X10
home_player_X11
away_player_X1
away_player_X2
away_player_X3
away_player_X4
away_player_X5
away_player_X6
away_player_X7
away_player_X8
away_player_X9
away_player_X10
away_player_X11
home_player_Y1
home_player_Y2
home_player_Y3
home_player_Y4
home_player_Y5
home_player_Y6
home_player_Y7
home_player_Y8
home_player_Y9
home_player_Y10
home_player_Y11
away_player_Y1
away_player_Y2
away_player_Y3
away_player_Y4
away_player_Y5
away_player_Y6
away_player_Y7
away_player_Y8
away_player_Y9
away_player_Y10
away_player_Y11
home_player_1
home_player_2
home_player_3
home_player_4
home_player_5
home_player_6
home_player_7
home_player_8
home_player_9
home_player_10
home_player_11
away_player_1
away_player_2
away_player_3
away_player_4
a

In [257]:
short_match = df_match.dropna(subset = ['foulcommit'])[['match_api_id','date','home_team_api_id', 'away_team_api_id', 'foulcommit', 'card', 'corner']]
short_team = df_team[['team_api_id', 'team_long_name']]
m1 = pd.merge(
    short_match, short_team,
    how = 'inner', left_on = 'home_team_api_id',
    right_on = 'team_api_id', suffixes = ("_x", "_y"))

m2 = pd.merge(
    m1, short_team,
    how = 'inner', left_on = 'away_team_api_id',
    right_on = 'team_api_id', suffixes = ('_x', '_y'))

In [258]:
m2.head()

,match_api_id,date,home_team_api_id,away_team_api_id,foulcommit,card,corner,team_api_id_x,team_long_name_x,team_api_id_y,team_long_name_y
0,489042,2008-08-17 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
1,839802,2010-08-16 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
2,1024845,2011-11-26 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
3,1229300,2012-12-26 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
4,1474518,2013-12-07 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United


In [259]:
fouls_teams = m2.rename(
    columns = {'team_long_name_x' : 'home_team_name', 'team_long_name_y' : 'away_team_name'}
)#.filter(regex = 'id|date|home|away|foul')

In [260]:
fouls_teams.head()

,match_api_id,date,home_team_api_id,away_team_api_id,foulcommit,card,corner,team_api_id_x,home_team_name,team_api_id_y,away_team_name
0,489042,2008-08-17 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
1,839802,2010-08-16 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
2,1024845,2011-11-26 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
3,1229300,2012-12-26 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United
4,1474518,2013-12-07 00:00:00,10260,10261,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<corner><value><stats><corners>1</corners></st...,10260,Manchester United,10261,Newcastle United


In [276]:
def unravel_fouls(index):
    xml = fouls_teams['foulcommit'].iloc[index]
    match_id = fouls_teams['match_api_id'].iloc[index]
    root = etree.fromstring(xml)
    unique_elems = []
    for child in root:
        if child[0][0].tag not in unique_elems:
            unique_elems += [child[0][0].tag]

        for child1 in child[1:]:
            if child1.tag not in unique_elems:
                unique_elems += [child1.tag]
    unique_elems

    d = {'match_id': []}
    for elem in unique_elems:
        d[elem] = []

    for i, child in enumerate(root):
        d['match_id'] += [match_id]
        d[child[0][0].tag] += [child[0][0].text]

        for child1 in child[1:]:
            d[child1.tag] += [child1.text]

        key_lengths = [len(d[x]) for x in list(d.keys())]
        for x in list(d.keys()):
            if len(d[x]) < max(key_lengths):
                d[x] += [None]

    foul_df = pd.DataFrame(d)
    foul_df = foul_df.fillna({'elapsed_plus':0})
    foul_df['elapsed'] = pd.to_numeric(foul_df['elapsed'])
    foul_df['elapsed_plus'] = pd.to_numeric(foul_df['elapsed_plus'])
    foul_df['elapsed'] += foul_df['elapsed_plus']
    foul_df = foul_df.rename({'id' : 'foul_id'})
    foul_df = foul_df.drop(['elapsed_plus', 'n', 'sortorder'], axis = 1)
    return foul_df

In [280]:
def unravel_cards(index):
    xml = fouls_teams['card'].iloc[index]
    match_id = fouls_teams['match_api_id'].iloc[index]
    root = etree.fromstring(xml)
    unique_elems = []
    for child in root:
        if child[0].tag not in unique_elems:
            unique_elems += [child[0].tag]
        if child[1][0].tag not in unique_elems:
            unique_elems += [child[1][0].tag]
        for child1 in child[2:]:
            if child1.tag not in unique_elems:
                unique_elems += [child1.tag]
    unique_elems

    d = {'match_id': []}
    for elem in unique_elems:
        d[elem] = []

    for i, child in enumerate(root):
        d['match_id'] += [match_id]
        d[child[0].tag] += [child[0].text]
        d[child[1][0].tag] += [child[1][0].text]

        for child1 in child[2:]:
            d[child1.tag] += [child1.text]

        key_lengths = [len(d[x]) for x in list(d.keys())]
        for x in list(d.keys()):
            if len(d[x]) < max(key_lengths):
                d[x] += [None]

    card_df = pd.DataFrame(d)
    card_df['elapsed'] = pd.to_numeric(card_df['elapsed'])
    card_df = card_df.rename({'id' : 'card_id'})
    card_df = card_df.drop(['n', 'sortorder'], axis = 1)
    return card_df

In [281]:
unravel_cards(0)

,match_id,comment,ycards,event_incident_typefk,elapsed,card_type,subtype,player1,team,type,id
0,489042,y,1,73,78,y,serious_fouls,24157,10260,card,379481
1,489042,y,1,73,82,y,serious_fouls,30362,10260,card,379503
2,489042,y,1,70,90,y,None,30829,10260,card,379547


In [275]:
xml = fouls_teams['card'].iloc[1]
root = etree.fromstring(xml)
print(etree.tostring(root, pretty_print = True).decode())

<card>
  <value>
    <comment>y</comment>
    <stats>
      <ycards>1</ycards>
    </stats>
    <event_incident_typefk>73</event_incident_typefk>
    <elapsed>18</elapsed>
    <card_type>y</card_type>
    <subtype>serious_fouls</subtype>
    <player1>24205</player1>
    <sortorder>2</sortorder>
    <team>10261</team>
    <n>255</n>
    <type>card</type>
    <id>1132271</id>
  </value>
  <value>
    <comment>y</comment>
    <stats>
      <ycards>1</ycards>
    </stats>
    <event_incident_typefk>77</event_incident_typefk>
    <elapsed>22</elapsed>
    <card_type>y</card_type>
    <subtype>shirt_pull</subtype>
    <player1>23579</player1>
    <sortorder>4</sortorder>
    <team>10261</team>
    <n>261</n>
    <type>card</type>
    <id>1132291</id>
  </value>
  <value>
    <comment>y</comment>
    <stats>
      <ycards>1</ycards>
    </stats>
    <event_incident_typefk>73</event_incident_typefk>
    <elapsed>52</elapsed>
    <card_type>y</card_type>
    <subtype>serious_fouls</subtype>
   

In [247]:
fouls1 = unravel_fouls(0)
cards1 = unravel_cards(0)
merge1 = pd.merge(
    fouls_teams, fouls1,
    how = 'inner', left_on = 'match_api_id', right_on = 'match_id'
).drop(['foulcommit'], axis = 1)


,match_api_id,date,home_team_api_id,away_team_api_id,team_api_id_x,home_team_name,team_api_id_y,away_team_name,foul_idx,match_id,foulscommitted,event_incident_typefk,elapsed,player2,player1,team,type,id,subtype
0,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,0,489042,1,37,1,32569,25518,10261,foulcommit,378824,None
1,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,1,489042,1,37,2,24157,30929,10261,foulcommit,378826,None
2,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,2,489042,1,37,3,24148,29581,10261,foulcommit,378830,None
3,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,3,489042,1,37,5,40565,30373,10260,foulcommit,378841,None
4,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,4,489042,1,320,10,30829,29581,10261,foulcommit,378894,pushing
5,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,5,489042,1,37,13,25518,24157,10260,foulcommit,378913,None
6,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,6,489042,1,37,19,30362,24228,10261,foulcommit,378974,None
7,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,7,489042,1,37,27,30360,24157,10260,foulcommit,379035,None
8,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,8,489042,1,37,27,38807,24154,10260,foulcommit,379044,None
9,489042,2008-08-17 00:00:00,10260,10261,10260,Manchester United,10261,Newcastle United,9,489042,1,18,33,32569,34574,10261,foulcommit,379100,shirt_pull


In [ ]:
df_match[['card', 'cross', 'corner', 'possession']].dropna()